In [1]:
import numpy as np
import findspark
from functions.required_functions import (
    readFile,
    normalize,
    transform,
    get_block_data,
    train,
    accuracy,
)
from pyspark import SparkContext
import time
import json
import matplotlib.pyplot as plt
findspark.init()

In [2]:

# ==============================
# EXPERIMENT ORCHESTRATOR
# =============================
def run_experiment(experiment_name, partitions=None,nodes=None,file_path="data/botnet_reduced_10k_l.csv",nIter=10,learning_rate=1.5,lambda_reg=0,):
    results = []
    # Experimento variando partitions
    if partitions is not None:
        for i, n_partitions in enumerate(partitions):
            print(f"[{i+1}/{len(partitions)}]")
            result = _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_partitions=n_partitions,)
            result["partitions"] = n_partitions
            results.append(result)
        # Guardar resultados en JSON
        with open(f"results/{experiment_name}.json", "w") as f:
            json.dump(results, f, indent=4)
        _plot_results(
            results,
            x_key="partitions",
            title="Execution Time vs Partitions",
            xlabel="Number of partitions",
            filename=f"results/{experiment_name}_execution_time_vs_partitions.png",
        )
        _plot_stacked_times(
            results,
            x_key="partitions",
            filename=f"results/{experiment_name}_stacked_times_vs_partitions.png",
        )
    # Experimento variando workers
    elif nodes is not None:
        for i, n_node in enumerate(nodes):
            print(f"[{i+1}/{len(nodes)}]")
            result = _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_nodes=n_node,)
            result["nodes"] = n_node
            results.append(result)
        # Guardar resultados en JSON
        with open(f"results/{experiment_name}.json", "w") as f:
            json.dump(results, f, indent=4)

        _plot_results(
            results,
            x_key="nodes",
            title="Execution Time vs Workers",
            xlabel="Number of workers",
            filename=f"results/{experiment_name}_execution_time_vs_workers.png",
        )

        _plot_stacked_times(
            results,
            x_key="nodes",
            filename=f"results/{experiment_name}_stacked_times_vs_workers.png",
        )
    return results


# ==============================
# SPARK CONTEXT LIFECYCLE
# ==============================
def _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_partitions=None,n_nodes="*",):
    sc = SparkContext(
        master=f"local[{n_nodes}]",
        appName="Experiment_4",
    )
    try:
        result = _run_training(file_path,nIter,learning_rate,lambda_reg,sc,n_partitions,)
    finally:
        sc.stop()
    return result
# ==============================
# TRAINING PIPELINE
# ==============================


def _run_training(
    file_path,
    nIter,
    learning_rate,
    lambda_reg,
    sc,
    n_partitions,
):
    # READ
    start = time.time()
    data = readFile(file_path, sc)
    read_time = time.time() - start
    # REPARTITION
    repartition_time = 0
    if n_partitions is not None:
        start = time.time()
        data = data.repartition(n_partitions).cache()
        # Fuerza shuffle
        data.count()
        repartition_time = time.time() - start

    # NORMALIZE
    start = time.time()
    data = normalize(data).cache()
    data.count()
    normalize_time = time.time() - start
    # TRAIN
    start = time.time()
    w, b = train(
        data,
        nIter,
        learning_rate,
        lambda_reg,
    )
    train_time = time.time() - start
    # ACCURACY
    start = time.time()
    acc = accuracy(w, b, data)
    acc_calc_time = time.time() - start
    print(f"Accuracy: {acc:.4f}")
    total_time = (
        read_time + repartition_time + normalize_time + train_time + acc_calc_time
    )
    print(f"Tiempo total experimento: {total_time:.4f}s")
    return {
        "read_time": read_time,
        "repartition_time": repartition_time,
        "normalize_time": normalize_time,
        "train_time": train_time,
        "acc_calc_time": acc_calc_time,
        "total_time": total_time,
    }

# ==============================
# PLOTTING
# ==============================
def _plot_results(results, x_key, title, xlabel, filename):
    x = [r[x_key] for r in results]
    y = [r["total_time"] for r in results]

    plt.figure()
    plt.plot(x, y, marker="o")
    plt.xlabel(xlabel)
    plt.ylabel("Execution Time (s)")
    plt.title(title)
    plt.savefig(filename)
    plt.close()


def _plot_stacked_times(results, x_key, filename):
    x_labels = [r[x_key] for r in results]

    steps = ["read_time", "repartition_time", "normalize_time", "train_time", "acc_calc_time"]
    colors = ["#FF9999", "#66B2FF", "#99FF99", "#FFCC66", "#C266FF"]

    bottoms = [0]*len(results)
    plt.figure(figsize=(10, 6))

    for step, color in zip(steps, colors):
        heights = [r[step] for r in results]
        plt.bar(x_labels, heights, bottom=bottoms, color=color, label=step)
        bottoms = [b + h for b, h in zip(bottoms, heights)]

    plt.xlabel(x_key)
    plt.ylabel("Time (s)")
    plt.title("Time per step (stacked)")
    plt.legend()
    plt.savefig(filename)
    plt.close()

Los experimentos los vamos a ejecutar 8*4 + 1 veces porque tengo una CPU de 8 hilos virtuales (4 cores) y vamos a ver qué pasa cuando saturamos * 4  la CPU tanto con partitions como con nodos

In [3]:
run_experiment(file_path="data/botnet_tot_syn_l.csv", partitions=list(range(1, 8*4+1)), experiment_name = "experiment_4")


[1/32]


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/24 23:26:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/24 23:27:07 WARN BlockManager: Task 13 already completed, not releasing lock for rdd_10_0


Iteration 0/10, Cost: 0.7818, Accuracy: 0.5995


Iteration 1/10, Cost: 0.4867, Accuracy: 0.7723


Iteration 2/10, Cost: 0.3667, Accuracy: 0.8509


...

Iteration 7/10, Cost: 0.2226, Accuracy: 0.9265


Iteration 8/10, Cost: 0.2160, Accuracy: 0.9281


Iteration 9/10, Cost: 0.2107, Accuracy: 0.9296


Accuracy: 0.9296
Tiempo total experimento: 228.9833s


[{'read_time': 0.7698707580566406,
  'repartition_time': 8.416632890701294,
  'normalize_time': 19.095187425613403,
  'train_time': 525.2271234989166,
  'acc_calc_time': 12.408176898956299,
  'total_time': 565.9169914722443,
  'partitions': 1},
 {'read_time': 0.03763246536254883,
  'repartition_time': 6.185769319534302,
  'normalize_time': 11.708154439926147,
  'train_time': 313.7169988155365,
  'acc_calc_time': 7.714228630065918,
  'total_time': 339.3627836704254,
  'partitions': 2},
 {'read_time': 0.031238794326782227,
  'repartition_time': 5.890549421310425,
  'normalize_time': 9.413437604904175,
  'train_time': 250.69016027450562,
  'acc_calc_time': 5.840321779251099,
  'total_time': 271.8657078742981,
  'partitions': 3},
 {'read_time': 0.03608870506286621,
  'repartition_time': 5.84993052482605,
  'normalize_time': 8.281589984893799,
  'train_time': 217.97598600387573,
  'acc_calc_time': 5.1145102977752686,
  'total_time': 237.25810551643372,
  'partitions': 4},
 {'read_time': 0.0

In [4]:
run_experiment(file_path="data/botnet_tot_syn_l.csv", nodes = list(range(1, 8*4+1)), experiment_name = "experiment_5")

[1/32]


26/02/25 01:33:33 WARN BlockManager: Task 27 already completed, not releasing lock for rdd_4_0


Iteration 0/10, Cost: 0.7663, Accuracy: 0.5800


Iteration 1/10, Cost: 0.4652, Accuracy: 0.7814


Iteration 2/10, Cost: 0.3559, Accuracy: 0.8539


...

Iteration 7/10, Cost: 0.2266, Accuracy: 0.9232


Iteration 8/10, Cost: 0.2193, Accuracy: 0.9255


Iteration 9/10, Cost: 0.2134, Accuracy: 0.9272


Accuracy: 0.9272
Tiempo total experimento: 197.5220s


[{'read_time': 0.022001028060913086,
  'repartition_time': 0,
  'normalize_time': 34.15796232223511,
  'train_time': 405.7666211128235,
  'acc_calc_time': 9.468997955322266,
  'total_time': 449.4155824184418,
  'nodes': 1},
 {'read_time': 0.009666919708251953,
  'repartition_time': 0,
  'normalize_time': 19.260850429534912,
  'train_time': 245.7368562221527,
  'acc_calc_time': 5.622877359390259,
  'total_time': 270.63025093078613,
  'nodes': 2},
 {'read_time': 0.011826038360595703,
  'repartition_time': 0,
  'normalize_time': 15.587289094924927,
  'train_time': 193.62066555023193,
  'acc_calc_time': 4.563708066940308,
  'total_time': 213.78348875045776,
  'nodes': 3},
 {'read_time': 0.011092185974121094,
  'repartition_time': 0,
  'normalize_time': 13.806471824645996,
  'train_time': 171.67264890670776,
  'acc_calc_time': 4.024739503860474,
  'total_time': 189.51495242118835,
  'nodes': 4},
 {'read_time': 0.01307821273803711,
  'repartition_time': 0,
  'normalize_time': 13.735690832138